In [1]:
import os 
import sys
from tqdm import tqdm
import importlib
import numpy as np
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

In [3]:
BATCH_SIZE = 256
T = 20
VOCAB = ['start', 'view', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 16
HIDDEN_DIM = 128

## Classifier Model

In [4]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense, Dropout, Activation, Multiply, Add, Lambda
from tensorflow.keras import regularizers

from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall


def build_classifier(batch_size, T, event_vocab_dim, emb_dim, hidden_dim, dropout_rate=0.25):
    # normal LSTM
    i_et = Input(batch_shape=(batch_size, None, event_vocab_dim), name='event_type')  # input of discrete feature event type
    i_ts = Input(batch_shape=(batch_size, None, 1), name='time_delta_in')  # input of continuous feature timestamp
    
    embed0 = Dense(emb_dim, name='dense_emb')(i_et) # dense matrix size: 6*16
    merged0 = tf.concat([embed0, i_ts], axis=2)
    
    hm = LSTM(hidden_dim,
          name='lstm_token',
          stateful=False,
          return_sequences=False, 
          kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(merged0)

    tm = LSTM(hidden_dim,
          name='lstm_time',
          stateful=False,
          return_sequences=False,
          kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          recurrent_initializer=tf.keras.initializers.RandomNormal(stddev=0.1),
          bias_initializer=tf.keras.initializers.RandomNormal(stddev=0.1))(merged0)
    
    token_time_comb = tf.keras.layers.concatenate([hm, tm], axis=1)
    
    dropped = Dropout(rate=dropout_rate)(token_time_comb)
    
    prob = Dense(1, 
             activation='sigmoid',
             name='final',
             kernel_initializer=tf.keras.initializers.TruncatedNormal(mean=0.0, stddev=0.1, seed=None),
             bias_initializer=tf.keras.initializers.Constant(value=0.1))(dropped)
        
    classifier = Model(
        inputs=[i_et, i_ts],
        outputs=prob)
        
    metrics = [
        BinaryAccuracy(name='accuracy'),
        Precision(name='precision'),
        Recall(name='recall'),
        AUC(num_thresholds=500, curve='PR', name='auc_pr'),
        AUC(num_thresholds=500, curve='ROC', name='auc_roc')
    ]
    
    classifier.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(lr=0.001),
        metrics=metrics)
    
    return classifier

In [5]:
classifier = build_classifier(batch_size=BATCH_SIZE,
                              T=T,
                              event_vocab_dim=EVENT_VOCAB_DIM,
                              emb_dim=EMB_DIM,
                              hidden_dim=HIDDEN_DIM,    
                              dropout_rate=0.25)

In [6]:
classifier.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
event_type (InputLayer)         [(256, None, 4)]     0                                            
__________________________________________________________________________________________________
dense_emb (Dense)               (256, None, 16)      80          event_type[0][0]                 
__________________________________________________________________________________________________
time_delta_in (InputLayer)      [(256, None, 1)]     0                                            
__________________________________________________________________________________________________
tf.concat (TFOpLambda)          (256, None, 17)      0           dense_emb[0][0]                  
                                                                 time_delta_in[0][0]          

## Load Saved TF Datasets

In [7]:
path_prefix = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/classifier_tf_datasets'

In [44]:
train_names_gs_100 = [
'1-to-100-gs-im',
'1-to-100-gs-up',
'1-to-100-gs-G0',
'1-to-100-gs-G1',
'1-to-100-gs-G2']

train_names_gs_1000 = [
'1-to-1000-gs-im',
'1-to-1000-gs-up',
'1-to-1000-gs-G0',
'1-to-1000-gs-G1',
'1-to-1000-gs-G2']

val_data_path_gs_100 = os.path.join(path_prefix, '1-to-100-gs-val.tf.data')

val_data_path_gs_1000 = os.path.join(path_prefix, '1-to-1000-gs-val.tf.data')

In [45]:
results_save_path_prefix = '/home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/experiments'

In [46]:
# For each dataset, run 10 trails; At each Trail, build a new model and train to get best metrics

TRIALS = 3

epochs = 20
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5
)

In [47]:
from tensorflow import TensorSpec
element_spec = ((TensorSpec(shape=(BATCH_SIZE, T, 4), dtype=tf.float64, name=None), 
                 TensorSpec(shape=(BATCH_SIZE, T, 1), dtype=tf.float64, name=None)),
                TensorSpec(shape=(BATCH_SIZE, 1), dtype=tf.float64, name=None))

## Experiment for 1-to-100-gs

In [48]:
val_dataset_gs_100 = tf.data.experimental.load(val_data_path_gs_100, element_spec)

In [49]:
for name in train_names_gs_100:
    dataset_path = os.path.join(path_prefix, name + '.tf.data')
    train_dataset = tf.data.experimental.load(dataset_path, element_spec)
    
    print('Processing Data:', dataset_path)
    ten_run_bests = []
    
    for run in range(TRIALS):        
        print('Trial:', run)
        
        classifier_run = build_classifier(batch_size=BATCH_SIZE,
                                          T=T,
                                          event_vocab_dim=EVENT_VOCAB_DIM,
                                          emb_dim=EMB_DIM,
                                          hidden_dim=HIDDEN_DIM,    
                                          dropout_rate=0.25)
        
        run_results = classifier_run.fit(train_dataset, 
                                        epochs=epochs,
                                        validation_data=val_dataset_gs_100,
                                        shuffle=True,
                                        callbacks=[early_stopping])
        
        run_history = run_results.history
        best_ind = np.argmin(run_history['val_loss'])
        run_best = {k : v_list[best_ind] for k, v_list in run_history.items()}
        
        ten_run_bests.append(run_best)
    
    df_ten_bests = pd.DataFrame(ten_run_bests)
    results_save_path = os.path.join(results_save_path_prefix, name + '_ten_bests.csv')
    df_ten_bests.to_csv(results_save_path)

Processing Data: /home/lun/project-basileus/multitype-sequence-generation-by-tlstm-gan/data/classifier_tf_datasets/1-to-100-gs-im.tf.data
Trial: 0
Epoch 1/20
946/946 [==============================] - 82s 84ms/step - loss: 0.0496 - accuracy: 0.9888 - precision: 0.4063 - recall: 0.0983 - auc_pr: 0.1766 - auc_roc: 0.8442 - val_loss: 0.0202 - val_accuracy: 0.9931 - val_precision: 0.9105 - val_recall: 0.3308 - val_auc_pr: 0.6800 - val_auc_roc: 0.9887
Epoch 2/20
946/946 [==============================] - 122s 130ms/step - loss: 0.0188 - accuracy: 0.9937 - precision: 0.8106 - recall: 0.4676 - auc_pr: 0.6835 - auc_roc: 0.9850 - val_loss: 0.0138 - val_accuracy: 0.9955 - val_precision: 0.8761 - val_recall: 0.6323 - val_auc_pr: 0.8253 - val_auc_roc: 0.9934
Epoch 3/20
946/946 [==============================] - 125s 132ms/step - loss: 0.0133 - accuracy: 0.9953 - precision: 0.8524 - recall: 0.6233 - auc_pr: 0.8188 - auc_roc: 0.9926 - val_loss: 0.0118 - val_accuracy: 0.9960 - val_precision: 0.9441 -

946/946 [==============================] - 125s 132ms/step - loss: 0.0069 - accuracy: 0.9976 - precision: 0.9145 - recall: 0.8317 - auc_pr: 0.9411 - auc_roc: 0.9955 - val_loss: 0.0089 - val_accuracy: 0.9971 - val_precision: 0.8553 - val_recall: 0.8468 - val_auc_pr: 0.9175 - val_auc_roc: 0.9957
Epoch 11/20
946/946 [==============================] - 125s 132ms/step - loss: 0.0061 - accuracy: 0.9980 - precision: 0.9306 - recall: 0.8586 - auc_pr: 0.9511 - auc_roc: 0.9964 - val_loss: 0.0098 - val_accuracy: 0.9970 - val_precision: 0.8532 - val_recall: 0.8430 - val_auc_pr: 0.9111 - val_auc_roc: 0.9875
Epoch 12/20
946/946 [==============================] - 125s 132ms/step - loss: 0.0057 - accuracy: 0.9980 - precision: 0.9211 - recall: 0.8644 - auc_pr: 0.9576 - auc_roc: 0.9971 - val_loss: 0.0087 - val_accuracy: 0.9973 - val_precision: 0.8742 - val_recall: 0.8474 - val_auc_pr: 0.9214 - val_auc_roc: 0.9924
Epoch 13/20
946/946 [==============================] - 125s 132ms/step - loss: 0.0053 - acc

1875/1875 [==============================] - 224s 120ms/step - loss: 0.0053 - accuracy: 0.9987 - precision: 0.9978 - recall: 0.9995 - auc_pr: 0.9997 - auc_roc: 0.9999 - val_loss: 0.0195 - val_accuracy: 0.9963 - val_precision: 0.7866 - val_recall: 0.8574 - val_auc_pr: 0.8475 - val_auc_roc: 0.9646
Epoch 8/20
1875/1875 [==============================] - 225s 120ms/step - loss: 0.0042 - accuracy: 0.9988 - precision: 0.9982 - recall: 0.9994 - auc_pr: 0.9998 - auc_roc: 0.9999 - val_loss: 0.0188 - val_accuracy: 0.9966 - val_precision: 0.8312 - val_recall: 0.8286 - val_auc_pr: 0.8512 - val_auc_roc: 0.9570
Epoch 9/20
1875/1875 [==============================] - 224s 119ms/step - loss: 0.0025 - accuracy: 0.9993 - precision: 0.9989 - recall: 0.9997 - auc_pr: 0.9999 - auc_roc: 1.0000 - val_loss: 0.0235 - val_accuracy: 0.9963 - val_precision: 0.7964 - val_recall: 0.8418 - val_auc_pr: 0.8222 - val_auc_roc: 0.9528
Epoch 10/20
1875/1875 [==============================] - 225s 120ms/step - loss: 0.0024

1875/1875 [==============================] - 225s 120ms/step - loss: 0.0038 - accuracy: 0.9987 - precision: 0.9992 - recall: 0.9982 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0089 - val_accuracy: 0.9968 - val_precision: 0.9761 - val_recall: 0.6892 - val_auc_pr: 0.9342 - val_auc_roc: 0.9960
Epoch 8/20
1875/1875 [==============================] - 226s 120ms/step - loss: 0.0034 - accuracy: 0.9988 - precision: 0.9992 - recall: 0.9983 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0065 - val_accuracy: 0.9979 - val_precision: 0.9618 - val_recall: 0.8180 - val_auc_pr: 0.9512 - val_auc_roc: 0.9964
Epoch 9/20
1875/1875 [==============================] - 226s 120ms/step - loss: 0.0030 - accuracy: 0.9989 - precision: 0.9992 - recall: 0.9986 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0071 - val_accuracy: 0.9978 - val_precision: 0.9726 - val_recall: 0.7986 - val_auc_pr: 0.9488 - val_auc_roc: 0.9915
Epoch 10/20
1875/1875 [==============================] - 226s 120ms/step - loss: 0.0026

1875/1875 [==============================] - 233s 122ms/step - loss: 0.0687 - accuracy: 0.9674 - precision: 0.9698 - recall: 0.9660 - auc_pr: 0.9899 - auc_roc: 0.9893 - val_loss: 0.0222 - val_accuracy: 0.9928 - val_precision: 0.8275 - val_recall: 0.3421 - val_auc_pr: 0.6141 - val_auc_roc: 0.9834
Epoch 2/20
1875/1875 [==============================] - 223s 119ms/step - loss: 0.0106 - accuracy: 0.9966 - precision: 0.9988 - recall: 0.9943 - auc_pr: 0.9998 - auc_roc: 0.9998 - val_loss: 0.0155 - val_accuracy: 0.9948 - val_precision: 0.8749 - val_recall: 0.5597 - val_auc_pr: 0.7964 - val_auc_roc: 0.9938
Epoch 3/20
1875/1875 [==============================] - 222s 119ms/step - loss: 0.0074 - accuracy: 0.9975 - precision: 0.9988 - recall: 0.9961 - auc_pr: 0.9999 - auc_roc: 0.9999 - val_loss: 0.0132 - val_accuracy: 0.9952 - val_precision: 0.9443 - val_recall: 0.5510 - val_auc_pr: 0.8470 - val_auc_roc: 0.9931
Epoch 4/20
1875/1875 [==============================] - 222s 119ms/step - loss: 0.0060 

1875/1875 [==============================] - 236s 126ms/step - loss: 0.0027 - accuracy: 0.9991 - precision: 0.9995 - recall: 0.9987 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0086 - val_accuracy: 0.9975 - val_precision: 0.9178 - val_recall: 0.8168 - val_auc_pr: 0.9272 - val_auc_roc: 0.9860
Epoch 13/20
1875/1875 [==============================] - 236s 126ms/step - loss: 0.0021 - accuracy: 0.9992 - precision: 0.9995 - recall: 0.9989 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0088 - val_accuracy: 0.9976 - val_precision: 0.9354 - val_recall: 0.8155 - val_auc_pr: 0.9257 - val_auc_roc: 0.9848
Epoch 14/20
1875/1875 [==============================] - 236s 126ms/step - loss: 0.0020 - accuracy: 0.9993 - precision: 0.9995 - recall: 0.9990 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0091 - val_accuracy: 0.9973 - val_precision: 0.8680 - val_recall: 0.8593 - val_auc_pr: 0.9218 - val_auc_roc: 0.9886
Trial: 1
Epoch 1/20
1875/1875 [==============================] - 228s 118ms/step - lo

1875/1875 [==============================] - 221s 118ms/step - loss: 0.0037 - accuracy: 0.9987 - precision: 0.9991 - recall: 0.9983 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0079 - val_accuracy: 0.9974 - val_precision: 0.9728 - val_recall: 0.7592 - val_auc_pr: 0.9392 - val_auc_roc: 0.9913
Epoch 9/20
1875/1875 [==============================] - 221s 118ms/step - loss: 0.0033 - accuracy: 0.9989 - precision: 0.9993 - recall: 0.9986 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0083 - val_accuracy: 0.9973 - val_precision: 0.9762 - val_recall: 0.7455 - val_auc_pr: 0.9404 - val_auc_roc: 0.9880
Epoch 10/20
1875/1875 [==============================] - 221s 118ms/step - loss: 0.0028 - accuracy: 0.9990 - precision: 0.9993 - recall: 0.9987 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0098 - val_accuracy: 0.9968 - val_precision: 0.9892 - val_recall: 0.6879 - val_auc_pr: 0.9364 - val_auc_roc: 0.9880
Epoch 11/20
1875/1875 [==============================] - 221s 118ms/step - loss: 0.002

1875/1875 [==============================] - 285s 149ms/step - loss: 0.0490 - accuracy: 0.9821 - precision: 0.9829 - recall: 0.9821 - auc_pr: 0.9965 - auc_roc: 0.9961 - val_loss: 0.0224 - val_accuracy: 0.9929 - val_precision: 0.7307 - val_recall: 0.4428 - val_auc_pr: 0.6106 - val_auc_roc: 0.9827
Epoch 2/20
1875/1875 [==============================] - 278s 148ms/step - loss: 0.0110 - accuracy: 0.9963 - precision: 0.9986 - recall: 0.9941 - auc_pr: 0.9999 - auc_roc: 0.9998 - val_loss: 0.0132 - val_accuracy: 0.9954 - val_precision: 0.8529 - val_recall: 0.6492 - val_auc_pr: 0.8306 - val_auc_roc: 0.9947
Epoch 3/20
1875/1875 [==============================] - 277s 148ms/step - loss: 0.0078 - accuracy: 0.9973 - precision: 0.9987 - recall: 0.9960 - auc_pr: 0.9999 - auc_roc: 0.9999 - val_loss: 0.0113 - val_accuracy: 0.9960 - val_precision: 0.8077 - val_recall: 0.7855 - val_auc_pr: 0.8812 - val_auc_roc: 0.9958
Epoch 4/20
1875/1875 [==============================] - 278s 148ms/step - loss: 0.0063 

Epoch 11/20
1875/1875 [==============================] - 277s 148ms/step - loss: 0.0024 - accuracy: 0.9991 - precision: 0.9994 - recall: 0.9989 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0093 - val_accuracy: 0.9973 - val_precision: 0.9524 - val_recall: 0.7636 - val_auc_pr: 0.9280 - val_auc_roc: 0.9870
Epoch 12/20
1875/1875 [==============================] - 277s 148ms/step - loss: 0.0020 - accuracy: 0.9992 - precision: 0.9994 - recall: 0.9990 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0091 - val_accuracy: 0.9977 - val_precision: 0.9587 - val_recall: 0.7980 - val_auc_pr: 0.9340 - val_auc_roc: 0.9823
Epoch 13/20
1875/1875 [==============================] - 277s 148ms/step - loss: 0.0017 - accuracy: 0.9994 - precision: 0.9995 - recall: 0.9992 - auc_pr: 1.0000 - auc_roc: 1.0000 - val_loss: 0.0075 - val_accuracy: 0.9979 - val_precision: 0.9003 - val_recall: 0.8812 - val_auc_pr: 0.9404 - val_auc_roc: 0.9891
Epoch 14/20
1875/1875 [==============================] - 276s 147ms/step 

In [51]:
TRIALS

3

## Experiment for 1-to-1000-gs

In [ ]:
val_dataset_gs_1000 = tf.data.experimental.load(val_data_path_gs_1000, element_spec)

In [ ]:
for name in train_names_gs_1000:
    dataset_path = os.path.join(path_prefix, name + '.tf.data')
    train_dataset = tf.data.experimental.load(dataset_path, element_spec)
    
    print('Processing Data:', dataset_path)
    ten_run_bests = []
    
    for run in range(TRIALS):
        print('Trial:', run)
        classifier_run = build_classifier(batch_size=BATCH_SIZE,
                                          T=T,
                                          event_vocab_dim=EVENT_VOCAB_DIM,
                                          emb_dim=EMB_DIM,
                                          hidden_dim=HIDDEN_DIM,    
                                          dropout_rate=0.25)
        
        run_results = classifier_run.fit(train_dataset, 
                                        epochs=epochs,
                                        validation_data=val_dataset_gs_1000,
                                        shuffle=True,
                                        callbacks=[early_stopping])
        
        run_history = run_results.history
        best_ind = np.argmin(run_history['val_loss'])
        run_best = {k : v_list[best_ind] for k, v_list in run_history.items()}
        
        ten_run_bests.append(run_best)
    
    df_ten_bests = pd.DataFrame(ten_run_bests)
    results_save_path = os.path.join(results_save_path_prefix, name + '_ten_bests.csv')
    df_ten_bests.to_csv(results_save_path)